<div class="alert alert-block alert-danger" style="color:black">
    <h2> This is an assessed practical</h2>
    The final part of the portfolio is to author the knowledge base for an expert system (chatbot) as described in activity 3. It is worth 100 marks.<br>
    You are given a file containing the set of questions and answers (<code>portfolio-questions-and-responses-2025.txt</code>), together with a jupyter notebook in which to develop your aiml file (<code>AIML_Selfchecker.ipynb</code>), and run it through a copy of the marking and feedback code prior to submission.
</div>

#  Workbook 9: Connecting the chatbot to other systems and data

Overview of activities and objectives of this workbook:

1. The first part of this workbook will how to connect the bot to the operating system and other sources of data.
    - We will set some predicates to allow the bot to use operating `<system>` tag-pairs to make calls and connecting your chatbot to external services, like date and time.
    - We will enable the bot to create web queries based on user input, with OS commands and wildcards.

2. The second part of this workbook gives you a chance to work on the final part of the portfolio, and make sure your testing toolchain is in place.
    - You should make sure you understand what you need to do for the final part of the portfolio.
    - You should make sure you know how to test your chatbot, using the toolchain for developing the coursework.

<div style="background:black;width:100%;height:10px"></div><br>

# Part 1: Communications between your bot and other programmes

AIML V2 comes with new tags such as `<sraix>` that send messages to other programmes either on the local machine, or running as a web service.

However even in AIML v1, there is the option to make a "system" call, which works in the same way as running a command from a terminal (mac/linux) or the "command tool" (windows).

<div class="alert alert-block alert-info" style="color:black"><h2>Activity 1: Setting/Getting Bot variables from outside the bot, and using system tags</h2>

Start by reading then running the 3 cells below which:
1. Create a bot.
2. Demonstrate how an external programme can tell a running bot to set some predicate values according to the computer it is hosted on.
3. Demonstrate how an external programme can  retrieve short term memory (predicates) from a running bot.

Then run the cells after those to check the values from inside the bot.
</div>

### Setting some bot predicates using python outside the bot

In [ ]:
import aiml
from IPython.display import HTML
import platform

# Create the kernel and learn AIML files
myChatbot = aiml.Kernel()

# use a standard python library to find out what operating system this code is running on
osname = platform.system()


# set some command  according to the operating system
if osname == "Darwin":  # Macos
    openUrlCommand = "open"
    getDateCommand = "date | cut -d' ' -f1-3,6"
    getTimeCommand = "date|cut -d' ' -f4,5"
elif osname == "Windows":
    openUrlCommand = "start"
    # ideally the next two commands would be date/T and time/T, which just retrieve current values
    # but the aiml interpreter change the / to \ on windows so they don't get sent through correctly
    # these versions should work fine - you can safely ignore the message about setting a new time
    getDateCommand = "date"
    getTimeCommand = "time"
else:  # linux
    print("There may not be a command-line command to open an url on your system.\n On linux it depends what you have installed")
    openUrlCommand = ""
    getDateCommand = "date | cut -d' ' -f1-3,6"
    getTimeCommand = "date|cut -d' ' -f4,5"


# set some values using the setPredicate() method
myChatbot.setPredicate("osname", osname)
myChatbot.setPredicate("getDateCommand", getDateCommand)
myChatbot.setPredicate("getTimeCommand", getTimeCommand)
myChatbot.setPredicate("openURLCommand", openUrlCommand)

### Querying values of bot predicates from outside the bot

In [ ]:
## In this cell python code asks the bot for values of variables (predicates) using the getPredicate() method


print("The bot has its variable osname set to " 
    + myChatbot.getPredicate("osname"))

print("The bot will get the date calling system command: "
    + myChatbot.getPredicate("getDateCommand")
)

print("The bot will get the time by calling system command: "
    + myChatbot.getPredicate("getTimeCommand")
)

print("The bot will use this command to open a url: "
    + myChatbot.getPredicate("openURLCommand")
)

### Next read, understand, then run the  cell below to write some categories to file

The categories contain the knowlege about  how to answer  these things in an interactive chat session.

The first one just retrieves a stored value and returns it.

The second and third categories  use `<system>` calls to the host computer, retrieving stored values to set the commands to be run.

You should then be able to ask the chatbot:
- what time it is.
- what date it is.
- what operating system it is running on.

In [ ]:
%%writefile "simple_extensions.aiml"
<aiml version="1.0.1" encoding="UTF-8">

<category>
    <pattern> WHAT OPERATING SYSTEM ARE YOU RUNNING ON</pattern>
    <template><get name="osname" /></template>
</category>

<category>
    <pattern> WHAT IS THE TIME </pattern>
    <template> 
          It is <system><get name = "getTimeCommand"/></system>
    </template>
</category>

<category>
    <pattern> WHAT IS THE DATE </pattern>
    <template> 
        Today is <system><get name="getDateCommand"/></system>
    </template>
</category>

</aiml>

In [ ]:
myChatbot.learn("data/simple_extensions.aiml")
myChatbot.verbose(True)

keepGoing = True
while keepGoing:
    nextInput = input("Enter your message >> ")
    if nextInput == "bye":
        keepGoing = False
    else:
        print(myChatbot.respond(nextInput))

<div class="alert alert-block alert-info" style="color:black"><h2>Activity 2: Encoding knowledge for a customer service chatbot</h2>

First <b>read</b>, then <b>run</b> the cell below to store some categories which make calls to external services.

Each one does the same three things in its template:

1. Uses a `<think>` pair to construct a variable called `runquery` by adding together:
    - The computer-specific command to open a url in a new tab.
    - The name of a web service.
    - The thing the user asked for - stored in `<star/>`.
2. Prints out a message and the hyperlink it hs constructed.
3. Uses a `<system>` tag-pair to ask the host computer to perform that action.

Then you can try these commands:
- Google Nelson
- Wikipedia Nelson
- Map of Nelson
- Images Nelson

**What does the number of results for Nelson Mandela tell you about how biassed the information on the web is??**


**On your own machine** these should open a new tab, and produce a hyperlink in the chat window.

**If you are using the csctcloud server** then:
- You will still get the hyperlink in the chat window, which should open a new tab in your browser when you click on it. 
- But for security reasons it's harder to (or Jim doesn't know how to) make a remote machine (csctcloud) open a tab in your browser.

</div>

In [ ]:
%%writefile "web_extensions.aiml"
<aiml version="1.0.1" encoding="UTF-8">

<category>
    <pattern> GOOGLE *</pattern>
    <template>
        <think>
            <set name="runquery"><get name="openURLCommand"/> http://www.google.co.uk/search?q=<star/></set>
        </think>
        If the tab does not open, here is a clickable link <get name="runquery"/>
        <system><get name="runquery"/></system>
    </template>
</category>

<category>
  <pattern> IMAGES *</pattern>
  <template>
        <think>
            <set name="runquery"><get name="openURLCommand"/> http://www.google.co.uk/images?q=<star/></set>
        </think>
        If the tab does not open, here is a clickable link <get name="runquery"/>
        <system><get name="runquery"/></system>
    </template>
</category>

<category>
    <pattern> MAP OF *</pattern>
    <template>
        <think>
            <set name="runquery"><get name="openURLCommand"/> http://www.google.co.uk/maps?q=<star/></set>
        </think>
        If the tab does not open, here is a clickable link <get name="runquery"/>
        <system><get name="runquery"/></system>
    </template>
</category>

<category>
    <pattern> WIKIPEDIA * </pattern>
    <template>
        <think>
            <set name="runquery"><get name="openURLCommand"/> https://en.wikipedia.org/wiki/<star/></set>
        </think>
        If the tab does not open, here is a clickable link <get name="runquery"/>
        <system><get name="runquery"/></system>
    </template>
</category>

<category>
    <pattern> CALCULATE * </pattern>
    <template>
        <think>
            <set name="runquery"><get name="openURLCommand"/> http://www.google.co.uk/search?q=<star/></set>
        </think>
        If the tab does not open, here is a clickable link <get name="runquery"/>
        <system><get name="runquery"/></system>
    </template>
</category>

</aiml>

In [ ]:
import aiml

myChatbot.learn("data/web_extensions.aiml")
myChatbot.verbose(True)

keepGoing = True
while keepGoing:
    nextInput = input("Enter your message >> ")
    if nextInput == "bye":
        keepGoing = False
    else:
        print(myChatbot.respond(nextInput))

<div style="background:black;width:100%;height:10px"></div><br>

# Part 2: Final portfolio submission - creating a knowledge-base for a chatbot

<div class="alert alert-block alert-info" style="color:black"><h2>Activity 3: A chatbot to answer the portfolio questions</h2>

<h3><b>100 marks</b> - using the marking scheme below</h3>

You should now have knowledge of all of the AIML concepts and tags that you need to complete the final part of the portfolio.

In this weeks folder you will find a file called `portfolio-questions-and-responses-2024.txt` which contains 45 questions and corresponding answers. The task is to author a single file AIML  code that can be loaded into a bot and correctly answer all 45 questions.

You will score more marks for using the knowledge representation language features supported by AIML to reduce the redundancy in  your knowledge base by using as few categories as possible.

We have provided an `AIML_Selfchecker.ipyb` for you to develop and test your chatbot. You can use it to test your AIML using exactly the same code as on the marking server.

1. Open the notebook `AIML_Selfchecker.ipyb` in this directory.
    - The first cell is for you to develop you aiml code in.
        - When you run that cell it writes your aiml to a file called `student.aiml`
        - **Do not change the name of the file produced (i.e. leave it as student.aiml)**

    - Subsequent cells in the notebook run a copy of the same code that is on the marking system.
        - **Note that every time the marking system runs it asks the question in a different order**.
        - But it does keep together the 3 questions that naturally follow on from a particular response.

    - It should give you output on screen and also produce two files:
        - "student-feedback" with a copy of what is shown on screen.
        - "student-responses.txt" which shows you eaach question in turn, what was expected, and what your bot produced, nicely aligned so you can check for typos, missing punctuation etc.

2. Using the ideas from week 9 and 10, edit the first cell (which produces "student.aiml") to contain categories encoding all the information you need to answer the set of 45 questions in `portfolio-questions-and-responses-2025.txt`.
    - We suggest you might start with 45 categories and get your bot correctly answering all the questions.

3. You will need to use some categories with `<that>` tag-pairs to answer three of the questions correctly, because they rely on the previous question to provide context.

4. Then use what you have learned about `<srai>` tags and wildcards to reduce the number of categories in your file - **this will get you more marks**.

5. For even more marks try including `<think>`, `<set>` and `<get>` tags and looking at the lecture notebook to see how `<condition>` tags work in AIML.

6. You **DO NOT** need any of the AIML topics covered in this week (`<system>` commands etc), only those from weeks 9 and 10.

7. Once you have completed your AIML you just need to submit the file `student.aiml` produced by the `AIML_Selfchecker.ipyb` notebook.

**Make sure your "student.aiml" file is somewhere that is automatically backed up (like your UWE onedrive).**

</div>

### Marking scheme



| 0-44                                   | 45-90                                  | 100                                            |
|----------------------------------------|----------------------------------------|------------------------------------------------|
| Incorrect responses to some questions. | Correct response to each question.     | Correct response to each question.             | 
|----------------------------------------|----------------------------------------|------------------------------------------------|
|                                        | AIMl language features are used well   |  Exceptional analysis of knowledge to be stored|
|                                        | to provide concise knowledge management|  and how to encode it concisely.               |
|                                        | Solutions that duplicate information   | Wide range of AIMl language features are used. |
|                                        | will get 65 marks.                     |                                                |
|----------------------------------------|----------------------------------------|------------------------------------------------|
| One mark per correct answer.           | 90-N marks,   where  10< N<45          | Fewer than 10 categories used.                 |
|                                        | is the number of AIML categories  used |                                                |
|                                        | to encode the knowledge base.          |                                                |


<div class="alert alert-block alert-success" style="color:black"><b>Save and close Jupyter:</b>
    <ol>
        <li>Use the jupyterlab functions to download your work (ask your tutor if you need help with this) and save it somewhere sensible so you can find it easily.</li>
        <li>Shutdown the notebook when you have finished with this tutorial (menu->file->close and shutdown notebook)</li>
    </ol>
</div